In [1]:
import pandas as pd
import numpy as np
import os
import requests
import zipfile
import io
import glob
import re
from datetime import datetime
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import datasets, linear_model

import sklearn
from sklearn import *
from sklearn.cross_validation import train_test_split
from sklearn.metrics import *

/Users/vikasshah/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/vikasshah/anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/vikasshah/anaconda/lib/python3.5/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", Deprecat

In [3]:
data_frame = pd.read_csv('CombinedData.csv')


/Users/vikasshah/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,19,49,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df = data_frame
df= df.dropna(thresh=int(len(df.columns)/2))

In [5]:
df.ix[:1,0:10].head(2)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade
0,10119623,NaN,12000.0,12000.0,12000.0,36 months,11.99%,398.52,B,B3
1,10224583,NaN,11100.0,11100.0,11100.0,36 months,14.98%,384.68,C,C3


In [6]:
## remove months from column term and it has only 3 null values, replace it with ffill method.

df['term']=df['term'].str.split(' ',2).str[1]
df['term'] = df['term'].fillna(method='ffill')
df['term'] = df['term'].astype(int)
## remove % from column int_rate column and it has only 3 null values, replace it with ffill method.
df['int_rate'] = df['int_rate'].str.split('%',1).str[0]
df['int_rate'] =df['int_rate'].astype(float)

## 
df['loan_amnt'] = df['loan_amnt'].astype(int)
df['funded_amnt'] = df['funded_amnt'].astype(int)
df['installment'] = df['installment'].astype(int)

df['num_tl_op_past_12m']= df['num_tl_op_past_12m'].fillna(0).astype(int)

## grade and sub-grade has no null values.

/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http:/

In [7]:
df.ix[:1,10:20].head(2)

,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc
0,LTC,10+ years,MORTGAGE,130000.0,Source Verified,Dec-2013,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN
1,Teacher,10+ years,MORTGAGE,90000.0,Not Verified,Dec-2013,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN


In [8]:
## remove years and other character from emp_length to make it numerical 
df['emp_length_clean'] = df.emp_length.str.replace('+','')
df['emp_length_clean'] = df.emp_length_clean.str.replace('<','')
df['emp_length_clean'] = df.emp_length_clean.str.replace('years','')
df['emp_length_clean'] = df.emp_length_clean.str.replace('year','')
df['emp_length_clean'] = df.emp_length_clean.str.replace('n/a','0')
df['emp_length_clean']=pd.to_numeric(df['emp_length_clean'], errors='ignore')
df['emp_length_clean'].fillna(0,inplace=True)
df['emp_length_clean'].astype(int)


/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

0          10
1          10
2           1
3          10
4           2
5           4
6          10
7          10
8           7
9          10
10          5
11          6
12         10
13          3
14          2
15         10
16          4
17          2
18         10
19          4
20         10
21          3
22          2
23         10
24          8
25          1
26          2
27         10
28         10
29         10
           ..
1319008     5
1319009     3
1319010     1
1319011     1
1319012     9
1319013    10
1319014     1
1319015     3
1319017     6
1319018     1
1319019     9
1319020     1
1319021     2
1319022     2
1319023    10
1319025     5
1319026     1
1319027     1
1319028     1
1319029     1
1319030     1
1319031     1
1319032     5
1319035     9
1319036    10
1319037     1
1319038     1
1319039     1
1319040     2
1319054     2
Name: emp_length_clean, dtype: int64

In [9]:
df.ix[:1,20:30].head(2)

,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths
0,debt_consolidation,Debt consolidation,809xx,CO,13.03,0.0,Nov-1997,715.0,719.0,1.0
1,other,Other,103xx,NY,3.73,1.0,Jun-2001,690.0,694.0,0.0


In [10]:
## dti, delinq_2yrs, fico_range_low, fico_range_high, earliest_cr_line has zero null values..
df.dti = df.dti.astype(int)
df.delinq_2yrs = df.delinq_2yrs.astype(int)
df.fico_range_low = df.fico_range_low.astype(int)
df.fico_range_high= df.fico_range_high.astype(int)
df.inq_last_6mths= df.inq_last_6mths.fillna(0)
df.inq_last_6mths = df.inq_last_6mths.astype(int)
df.earliest_cr_line = df.earliest_cr_line.fillna(method='ffill')

/Users/vikasshah/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [11]:
df.ix[:1,30:40].head(2)

,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv
0,NaN,NaN,9.0,0.0,10805.0,67%,19.0,f,0.0,0.0
1,16.0,NaN,9.0,0.0,6619.0,66.2%,12.0,f,0.0,0.0


In [12]:
## fill na
## mths_since_last_delinq has 646467 null values.. (do not think we need this for FE)
df.mths_since_last_delinq = df.mths_since_last_delinq.fillna(0).astype(int)

## mths_since_last_record has 1088168 null values.. (do not think we need this for FE)
df.mths_since_last_record = df.mths_since_last_record.fillna(0).astype(int)

##open_acc has 1088168 null values.. (do not think we need this for FE)
df.open_acc = df.open_acc.fillna(0).astype(int)

## pub_rec, revol_bal,total_acc has no null values
df.pub_rec = df.pub_rec.astype(int)
df.total_acc = df.total_acc.astype(int)
df.revol_bal = df.revol_bal.astype(int)
df.out_prncp = df.out_prncp.astype(int)
df.out_prncp_inv = df.out_prncp_inv.astype(int)

df['revol_util'] = df['revol_util'].str.split('%',1).str[0].astype(float)
## mean revol_util 
df['revol_util']=df['revol_util'].fillna((df.revol_util.mean())).astype(int)


/Users/vikasshah/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://

In [13]:
df.ix[:1,40:50].head(2)

,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d
0,14346.479050,14346.48,12000.0,2346.48,0.0,0.0,0.0,Jan-2017,398.28,NaN
1,13575.640006,13575.64,11100.0,2475.64,0.0,0.0,0.0,Feb-2016,4343.32,NaN


In [14]:
## total_pymnt,total_pymnt_inv
df.total_pymnt = df.total_pymnt.fillna(0).astype(float)
df.total_pymnt_inv = df.total_pymnt_inv.fillna(0).astype(float)
df.total_rec_prncp = df.total_rec_prncp.fillna(0).astype(float)
df.total_rec_int = df.total_rec_int.astype(float)
df.total_rec_late_fee = df.total_rec_late_fee.astype(float)
df.recoveries = df.recoveries.astype(float)
df.collection_recovery_fee = df.collection_recovery_fee.astype(float)
df.last_pymnt_d = df.last_pymnt_d.fillna(method='ffill')
df.last_pymnt_amnt = df.last_pymnt_amnt.fillna(0).astype(float)
df.next_pymnt_d = df.next_pymnt_d.fillna(method='ffill') 


/Users/vikasshah/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [15]:
df.ix[:1,50:60].head(2)

,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint
0,Mar-2017,729.0,725.0,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN
1,Aug-2016,724.0,720.0,0.0,16.0,1.0,INDIVIDUAL,NaN,NaN,NaN


In [16]:
df.last_credit_pull_d = df.last_credit_pull_d.fillna(method='ffill')
df.last_fico_range_high = df.last_fico_range_high.astype(int)
df.last_fico_range_low = df.last_fico_range_low.astype(int)
df.collections_12_mths_ex_med = df.collections_12_mths_ex_med.fillna(0).astype(int)
df.mths_since_last_major_derog = df.mths_since_last_major_derog.fillna(0).astype(int)
df.policy_code = df.policy_code.astype(int)
df.application_type = df.application_type.fillna('unknown')
df.annual_inc_joint = df.annual_inc_joint.fillna(0)
df.dti_joint = df.dti_joint.fillna(0)
df.verification_status_joint = df.verification_status_joint.fillna(0)



/Users/vikasshah/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [17]:
df.ix[:1,60:70].head(2)

,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util
0,0.0,0.0,327264.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,353402.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df.acc_now_delinq = df.acc_now_delinq.astype(int)
df.tot_coll_amt = df.tot_coll_amt.fillna(0).astype(int)
df.tot_cur_bal = df.tot_cur_bal.fillna(0).astype(int)
df.open_acc_6m = df.open_acc_6m.fillna(0).astype(int)
df.open_il_6m = df.open_il_6m.fillna(0).astype(int)
df.open_il_12m =df.open_il_12m.fillna(0).astype(int)
df.open_il_24m = df.open_il_24m.fillna(0).astype(int)
df.mths_since_rcnt_il = df.mths_since_rcnt_il.fillna(0).astype(int)
df.total_bal_il =df.total_bal_il.fillna(0).astype(int)
df.il_util= df.il_util.fillna(0).astype(int)
df.num_sats = df.num_sats.fillna(0).astype(int)


/Users/vikasshah/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [19]:
df.ix[:,70:80].head(2)

,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal
0,NaN,NaN,NaN,NaN,16200.0,NaN,NaN,NaN,4.0,36362.0
1,NaN,NaN,NaN,NaN,10000.0,NaN,NaN,NaN,2.0,39267.0


In [20]:
df.total_rev_hi_lim =df.total_rev_hi_lim.fillna(0).astype(float)
df.acc_open_past_24mths = df.acc_open_past_24mths.fillna(0).astype(int)
df.avg_cur_bal = df.avg_cur_bal.fillna(0).astype(float)
df.mths_since_recent_bc =df.mths_since_recent_bc.fillna(0).astype(int)

/Users/vikasshah/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [21]:
df.ix[:,80:90].head(2)

df.bc_util =df.bc_util.fillna(0).astype(int)
df.delinq_amnt = df.delinq_amnt.fillna(0).astype(int)
df.mths_since_recent_bc_dlq =df.mths_since_recent_bc_dlq.fillna(0).astype(int)
df.mths_since_recent_inq = df.mths_since_recent_inq.fillna(0).astype(int)
df.mths_since_recent_revol_delinq = df.mths_since_recent_revol_delinq.fillna(0)
df.tax_liens = df.tax_liens.astype(int)
df.tot_hi_cred_lim = df.tot_hi_cred_lim.fillna(0).astype(float)
df.total_il_high_credit_limit = df.total_il_high_credit_limit.fillna(0).astype(float)

df['inq_fi']=df['inq_fi'].fillna((df.inq_fi.mean())).astype(int)

/Users/vikasshah/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
del df['bc_open_to_buy']
del df['chargeoff_within_12_mths']
del df['mo_sin_old_il_acct']
del df['mo_sin_old_rev_tl_op']
del df['mo_sin_rcnt_rev_tl_op']
del df['mo_sin_rcnt_tl']
del df['mort_acc']
del df['mths_since_recent_bc']


del df['num_accts_ever_120_pd']
del df['num_actv_bc_tl']
del df['num_actv_rev_tl']
del df['num_bc_sats']
del df['num_bc_tl']
del df['num_il_tl']
del df['num_op_rev_tl']
del df['num_rev_accts']
del df['num_rev_tl_bal_gt_0']
del df['num_tl_120dpd_2m']
del df['num_tl_30dpd']
del df['num_tl_90g_dpd_24m']


del df['pct_tl_nvr_dlq']
del df['percent_bc_gt_75']
del df['pub_rec_bankruptcies']
del df['total_bal_ex_mort']

del df['total_bc_limit']

del df['id']
del df['member_id']
del df['emp_title']
del df['desc']
del df['max_bal_bc']
del df['all_util']
#del df['inq_fi']
del df['total_cu_tl']
del df['verification_status_joint']
del df['initial_list_status']
del df['inq_last_12m']
del df['total_bal_il']
del df['il_util']
del df['open_rv_12m']
del df['mths_since_recent_revol_delinq']
del df['mths_since_recent_bc_dlq']
del df['open_acc_6m']
del df['open_il_6m']
del df['open_il_12m']
del df['open_il_24m']
del df['mths_since_rcnt_il']


del df['open_rv_24m']
del df['mths_since_last_record'] 
del df['mths_since_last_major_derog']
del df['annual_inc_joint']
del df['dti_joint']
del df['next_pymnt_d']
del df['title']


del df['num_sats']
del df['acc_open_past_24mths']
#del  df['pub_rec']
del df['avg_cur_bal']

In [23]:
cor = df.corr()
cor.loc[:,:] = np.tril(cor, k=-1) 
cor = cor.stack()
cor[(cor > 0.55) | (cor < -0.55)]  

funded_amnt              loan_amnt               0.999624
funded_amnt_inv          loan_amnt               0.998665
                         funded_amnt             0.999123
installment              loan_amnt               0.946476
                         funded_amnt             0.946997
                         funded_amnt_inv         0.945943
fico_range_high          fico_range_low          1.000000
total_acc                open_acc                0.703382
out_prncp_inv            out_prncp               0.999997
total_pymnt              loan_amnt               0.630559
                         funded_amnt             0.630215
                         funded_amnt_inv         0.628587
                         installment             0.633237
total_pymnt_inv          loan_amnt               0.630774
                         funded_amnt             0.630517
                         funded_amnt_inv         0.630605
                         installment             0.633372
              

In [26]:
df.drop(['funded_amnt','installment','total_rev_hi_lim','funded_amnt_inv','funded_amnt','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','collection_recovery_fee','last_pymnt_amnt','last_fico_range_low','last_fico_range_high','tot_hi_cred_lim'],1,inplace=True)

ValueError: labels ['funded_amnt' 'installment' 'total_rev_hi_lim' 'funded_amnt_inv'
 'funded_amnt' 'out_prncp_inv' 'total_pymnt' 'total_pymnt_inv'
 'total_rec_prncp' 'total_rec_int' 'collection_recovery_fee'
 'last_pymnt_amnt' 'last_fico_range_low' 'last_fico_range_high'
 'tot_hi_cred_lim'] not contained in axis

In [28]:
cor = df.corr()
cor.loc[:,:] = np.tril(cor, k=-1) 
cor = cor.stack()
cor[(cor > 0.55) | (cor < -0.55)]

fico_range_high  fico_range_low    1.000000
total_acc        open_acc          0.703382
bc_util          revol_util        0.765500
tax_liens        pub_rec           0.715262
dtype: float64

In [43]:
df.earliest_cr_line_yr =pd.Series(df['earliest_cr_line'].str[4:]).astype(int)
df.last_credit_pull_d_yr = pd.Series(df['last_credit_pull_d'].str[4:]).astype(int)


df['credit_age'] = df.last_credit_pull_d_yr - df.earliest_cr_line_yr


/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [30]:
#df['verification_status'] = pd.Categorical.from_array(df.verification_status).labels


In [31]:
df['meanfico'] = (df.fico_range_low + df.fico_range_high)/2
df['addr_state'] = pd.Categorical.from_array(df.addr_state).labels
df['grade'] = pd.Categorical.from_array(df.grade).labels
df['home_ownership'] = pd.Categorical.from_array(df.home_ownership).labels
df['verification_status'] = pd.Categorical.from_array(df.verification_status).labels
df['loan_status'] = pd.Categorical.from_array(df.loan_status).labels
df['purpose']= pd.Categorical.from_array(df.purpose).labels

/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: Categorical.from_array is deprecated, use Categorical instead
  from ipykernel import kernelapp as app
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: 'labels' is deprecated. Use 'codes' instead
  from ipykernel import kernelapp as app
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [32]:
cor = df.corr()
cor.loc[:,:] = np.tril(cor, k=-1) 
cor = cor.stack()
cor[(cor > 0.55) | (cor < -0.55)]

grade            int_rate           0.955327
fico_range_high  fico_range_low     1.000000
total_acc        open_acc           0.703382
bc_util          revol_util         0.765500
tax_liens        pub_rec            0.715262
meanfico         fico_range_low     1.000000
                 fico_range_high    1.000000
dtype: float64

In [33]:
df.drop(['fico_range_high','fico_range_low'],1,inplace=True)

/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [34]:
#sub_grade
df['sub_grade']= pd.Categorical.from_array(df.sub_grade).labels

/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: Categorical.from_array is deprecated, use Categorical instead
  from ipykernel import kernelapp as app
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: 'labels' is deprecated. Use 'codes' instead
  from ipykernel import kernelapp as app
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [37]:
df2 = df[['loan_amnt','sub_grade','purpose','term','pub_rec','total_acc','int_rate','grade','inq_fi','emp_length_clean','home_ownership','annual_inc','verification_status','loan_status','addr_state',
'dti','delinq_2yrs','credit_age','inq_last_6mths','mths_since_last_delinq','open_acc','revol_util','out_prncp','recoveries','bc_util','meanfico']]

In [38]:
df2.isnull().any()

loan_amnt                 False
sub_grade                 False
purpose                   False
term                      False
pub_rec                   False
total_acc                 False
int_rate                  False
grade                     False
inq_fi                    False
emp_length_clean          False
home_ownership            False
annual_inc                False
verification_status       False
loan_status               False
addr_state                False
dti                       False
delinq_2yrs               False
credit_age                False
inq_last_6mths            False
mths_since_last_delinq    False
open_acc                  False
revol_util                False
out_prncp                 False
recoveries                False
bc_util                   False
meanfico                  False
dtype: bool

In [39]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
boston = load_boston()
X_train =  pd.DataFrame()


X_train = df2[['loan_amnt','term','sub_grade','purpose','pub_rec','grade','revol_util','inq_fi','emp_length_clean','home_ownership','annual_inc','verification_status','loan_status','addr_state',
'dti','delinq_2yrs','credit_age','inq_last_6mths','mths_since_last_delinq','open_acc','out_prncp','recoveries','bc_util','meanfico']]
y_train = pd.DataFrame()
y_train = df["int_rate"]


lr = LinearRegression()
rfe = RFE(lr, n_features_to_select=1)
rfe.fit(X_train,y_train)
 
print ("Features sorted by their rank:")
ranking_out = zip(map(lambda x: round(x, 4), rfe.ranking_), X_train.columns)
print (sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), X_train.columns)))

Features sorted by their rank:
[(1, 'sub_grade'), (2, 'loan_status'), (3, 'inq_fi'), (4, 'verification_status'), (5, 'pub_rec'), (6, 'inq_last_6mths'), (7, 'delinq_2yrs'), (8, 'grade'), (9, 'term'), (10, 'open_acc'), (11, 'emp_length_clean'), (12, 'bc_util'), (13, 'purpose'), (14, 'revol_util'), (15, 'home_ownership'), (16, 'meanfico'), (17, 'credit_age'), (18, 'mths_since_last_delinq'), (19, 'addr_state'), (20, 'recoveries'), (21, 'dti'), (22, 'out_prncp'), (23, 'annual_inc'), (24, 'loan_amnt')]


In [44]:
from sklearn.metrics import r2_score
import random

Train_Test_Data1 = pd.DataFrame()
Train_Test_Data1 =df2[["loan_amnt","sub_grade","purpose","term","pub_rec","emp_length_clean","home_ownership","annual_inc","verification_status","addr_state","dti","delinq_2yrs","inq_last_6mths","mths_since_last_delinq","open_acc","revol_util","meanfico"]]


Predcit_Data1 =  pd.DataFrame()
Predcit_Data1 = df["int_rate"]


#############################################################

from sklearn.metrics import r2_score

X_train, x_test, y_train, y_test = train_test_split(Train_Test_Data1,Predcit_Data1,random_state=0,train_size = 0.8)

In [41]:

from sklearn.metrics import *

lm=linear_model.LinearRegression()
lm.fit(X_train,y_train)
print("Coefficientis:",lm.coef_)
print("Intercept is",lm.intercept_)

test_pred = lm.predict(X_train)
RMSE = np.sqrt(mean_squared_error(test_pred, y_train ))
print("TrainData Rmse",RMSE)

### on test data
test_pred = lm.predict(x_test)
MAE = mean_absolute_error(y_test,test_pred)
print("R-Square",r2_score(y_test,test_pred))
MSE = mean_squared_error(y_test,test_pred)
MedianAE = median_absolute_error(y_test,test_pred)
MAPE = np.mean(np.abs((y_test - test_pred) / y_test)) * 100
RMSE = np.sqrt(mean_squared_error(y_test, test_pred))
print(MAE) 
print(MSE)
print(MAPE) 
print(RMSE) 

Coefficientis: [ -4.43785657e-08  -4.78167698e-03   6.99138851e-01  -4.51964545e-03
  -3.50053678e-02   1.74719994e-02  -1.73202399e-03   7.99712076e-02
   3.63664526e-03   7.47866818e-04  -2.85082801e-07   5.50154351e-02
   1.17128665e-01  -1.76269014e-04  -3.13837395e-05  -1.43331587e-02
   1.03624916e-03   3.82578279e-02  -6.15474105e-04  -4.82294507e-03
  -5.07882757e-06   4.50011706e-05   3.98326542e-03   1.61253996e-03]
Intercept is 4.34308030599
TrainData Rmse 0.901219935224


NameError: name 'x_test' is not defined

In [83]:
random_forest = RandomForestRegressor(n_jobs=2)
random_forest.fit(X_train, y_train)
y_pred=random_forest.predict(x_test)

print("Mean Absolute Error is ",mean_absolute_error(y_test,y_pred))
#print("Mean Absolute Percentage Error is ",mean_absolute_percentage_error(Y_train,y_pred))
print("Root Mean Squared Error ",np.sqrt(mean_squared_error(y_test,y_pred)))
print("Accuracy of the model is ",r2_score(y_test,y_pred)) 

MSE = mean_squared_error(y_test,y_pred)
MedianAE = median_absolute_error(y_test,y_pred)
MAPE = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
print(MAE)
print(MSE)
print(MAPE)
print(RMSE)

Mean Absolute Error is  0.353034433473
Root Mean Squared Error  0.623731819012
Accuracy of the model is  0.981309784573
0.658173967013
0.389041382048
2.572155929675441
0.623731819012


In [ ]:
##KNN

knn = KNeighborsRegressor(n_neighbors=3)
    # Fit the model on the training data.
model = knn.fit(X_train,y_train)
y_pred=knn.predict(x_test)
MSE = mean_squared_error(y_test,y_pred)
MedianAE = median_absolute_error(y_test,y_pred)
MAPE = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
print(MAE)
print(MSE)
print(MAPE)
print(RMSE)

In [ ]:
#### Neural Network

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(x_test)

nn_reg=MLPRegressor(hidden_layer_sizes=(20,10,20))
nn_reg.fit(X_train,y_train)
predictions = nn_reg.predict(x_test)
y_pred=predictions


print ("Neural Network Mean Absolute Error:",mean_absolute_error(y_test,y_pred)*100)
print("Neural Network Root Mean Squared Error:",np.sqrt(mean_squared_error(y_test,y_pred)))
print("Neural Network R2 Score:",r2_score(y_test,y_pred))


In [42]:
df2.to_csv('PredictionWithAllData.csv',index=None)